In [ ]:
import os
os.chdir('../ppo_balancer/ppo_balancer/')

In [21]:
import argparse
import logging
from typing import Tuple
import time

import gin
import gymnasium as gym
import numpy as np
import upkie.envs
from envs import make_ppo_balancer_env
from settings import EnvSettings, PPOSettings, TrainingSettings
from stable_baselines3 import PPO
from upkie.utils.raspi import configure_agent_process, on_raspi
from upkie.utils.robot_state import RobotState
from upkie.utils.robot_state_randomization import RobotStateRandomization

upkie.envs.register()

/Users/jules/miniconda3/envs/ppo_balancer/lib/python3.12/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieGroundVelocity-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/jules/miniconda3/envs/ppo_balancer/lib/python3.12/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieServoPositions-v4 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/jules/miniconda3/envs/ppo_balancer/lib/python3.12/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieServoTorques-v4 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/jules/miniconda3/envs/ppo_balancer/lib/python3.12/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieServos-v4 already in registry.
  logger.w

In [26]:
def get_sagittal_force(intensity: float, duration=None):

    mass = 5.34  # mass in [kg]
    force = intensity*mass*9.81  # Force in [N]
    if duration is not None and duration > 1e9:
        force = 0.0
    return force

In [27]:
def run_policy(env: gym.Wrapper, policy) -> None:
    """!
    Run the policy on a given environment.

    @param env Upkie environment, wrapped by the agent.
    @param policy MLP policy to follow.
    """

    torso_force_in_world = np.zeros(3)
    bullet_action = {
        "external_forces": {
            "torso": {
                "force": torso_force_in_world,
                "local": False,
            }
        }
    }
    observation, _ = env.reset()
    gain = np.array([10.0, 1.0, 0.0, 0.1])

    # Values of intensity sagitall force that is apply
    intensity_force = np.arange(0.0, 0.1, 0.01)
    isFall = False

    magnitude = get_sagittal_force(intensity_force[0])
    previous_magnitude = magnitude

    torso_force_in_world[0] = magnitude

    best_magnitude = 0.0

    index_intensity = 0
    start = time.time_ns()



    action = np.zeros(env.action_space.shape)
    observation, info = env.reset()
    reward = 0.0
    while isFall==False and index_intensity < intensity_force.shape[0]:
        
        action, _ = policy.predict(observation, deterministic=True)
        now = time.time_ns()
        duration = now-start
        magnitude = get_sagittal_force(intensity_force[index_intensity],duration = duration)
        
        torso_force_in_world[0] = magnitude
        env.bullet_extra(bullet_action)  # call before env.step

        observation, reward, terminated, truncated, info = env.step(action)
        if terminated or truncated or duration > 3e9:
            if terminated or truncated:
                print('est tombé')
                isFall = True
                best_magnitude = get_sagittal_force(
                    intensity_force[index_intensity-1])
            start = time.time_ns()
            if index_intensity < intensity_force.shape[0]:
                index_intensity += 1
            torso_force_in_world[0] = get_sagittal_force(
                intensity_force[index_intensity])
            print(f'Monitoring : intensity:{intensity_force[index_intensity]}')
            observation, _ = env.reset()
    return best_magnitude

In [28]:
def main(policy_path: str, training: bool) -> None:
    """!
    Load environment and policy, and run the latter on the former.

    @param policy_path Path to policy parameters.
    @param training If True, add training noise and domain randomization.
    """
    env_settings = EnvSettings()
    init_state = None
    if training:
        training_settings = TrainingSettings()
        init_state = RobotState(
            randomization=RobotStateRandomization(
                **training_settings.init_rand
            ),
        )
    with gym.make(
        env_settings.env_id,
        frequency=env_settings.agent_frequency,
        init_state=init_state,
        max_ground_velocity=env_settings.max_ground_velocity,
        regulate_frequency=True,
        spine_config=env_settings.spine_config,
    ) as velocity_env:
        env = make_ppo_balancer_env(
            velocity_env,
            env_settings,
            training=training,
        )
        ppo_settings = PPOSettings()
        policy = PPO(
            "MlpPolicy",
            env,
            policy_kwargs={
                "net_arch": {
                    "pi": ppo_settings.net_arch_pi,
                    "vf": ppo_settings.net_arch_vf,
                },
            },
            verbose=0,
        )
        policy.set_parameters(policy_path)
        run_policy(env, policy)

In [29]:
if on_raspi():
        configure_agent_process()

policy_path = f"policy/params.zip"
# Configuration
config_path = f"policy/operative_config.gin"
logging.info("Loading policy configuration from %s", config_path)
gin.parse_config_file(config_path)

try:
    main(policy_path, training=False)
except KeyboardInterrupt:
    logging.info("Caught a keyboard interrupt")


[2024-11-30 17:28:48,484] [info] Loading policy configuration from policy/operative_config.gin (2856064887.py:7)
/Users/jules/miniconda3/envs/ppo_balancer/lib/python3.12/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.bullet_extra to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.bullet_extra` for environment variables or `env.get_wrapper_attr('bullet_extra')` that will search the reminding wrappers.
  logger.warn(
[2024-11-30 17:28:48,527] [warning] UpkieGroundVelocity rate limiter is late by -6.000000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:48,769] [warning] UpkieGroundVelocity rate limiter is late by -15.000000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:48,821] [warning] UpkieGroundVelocity rate limiter is late by -1.800000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:48,837] [warning] UpkieGroundVelocity rate limiter is late by -2.700000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:49,00

Monitoring : intensity:0.01


[2024-11-30 17:28:51,760] [warning] UpkieGroundVelocity rate limiter is late by -1.700000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:51,777] [warning] UpkieGroundVelocity rate limiter is late by -1.200000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:51,796] [warning] UpkieGroundVelocity rate limiter is late by -2.600000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:51,844] [warning] UpkieGroundVelocity rate limiter is late by -0.700000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:51,878] [warning] UpkieGroundVelocity rate limiter is late by -2.400000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:51,897] [warning] UpkieGroundVelocity rate limiter is late by -2.300000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:51,913] [warning] UpkieGroundVelocity rate limiter is late by -7.500000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:52,510] [warning] UpkieGroundVelocity rate limiter is late by -0.600000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:52,592] [warning] UpkieGroundVelocity rate lim

Monitoring : intensity:0.02


[2024-11-30 17:28:54,810] [warning] UpkieGroundVelocity rate limiter is late by -0.700000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:54,844] [warning] UpkieGroundVelocity rate limiter is late by -1.000000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:55,075] [warning] UpkieGroundVelocity rate limiter is late by -1.200000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:55,109] [warning] UpkieGroundVelocity rate limiter is late by -1.600000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:55,127] [warning] UpkieGroundVelocity rate limiter is late by -0.600000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:55,143] [warning] UpkieGroundVelocity rate limiter is late by -0.700000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:55,180] [warning] UpkieGroundVelocity rate limiter is late by -2.600000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:55,192] [warning] UpkieGroundVelocity rate limiter is late by -0.800000 [ms] (rate_limiter.py:106)
[2024-11-30 17:28:55,225] [warning] UpkieGroundVelocity rate lim

...

[2024-11-30 17:29:09,993] [warning] UpkieGroundVelocity rate limiter is late by -0.500000 [ms] (rate_limiter.py:106)
[2024-11-30 17:29:10,026] [warning] UpkieGroundVelocity rate limiter is late by -1.300000 [ms] (rate_limiter.py:106)
[2024-11-30 17:29:10,060] [warning] UpkieGroundVelocity rate limiter is late by -1.700000 [ms] (rate_limiter.py:106)
[2024-11-30 17:29:10,108] [warning] UpkieGroundVelocity rate limiter is late by -1.100000 [ms] (rate_limiter.py:106)
[2024-11-30 17:29:10,125] [warning] UpkieGroundVelocity rate limiter is late by -0.700000 [ms] (rate_limiter.py:106)
[2024-11-30 17:29:10,143] [warning] UpkieGroundVelocity rate limiter is late by -1.200000 [ms] (rate_limiter.py:106)
[2024-11-30 17:29:10,159] [warning] UpkieGroundVelocity rate limiter is late by -0.800000 [ms] (rate_limiter.py:106)
[2024-11-30 17:29:10,208] [warning] UpkieGroundVelocity rate limiter is late by -0.700000 [ms] (rate_limiter.py:106)
[2024-11-30 17:29:10,225] [warning] UpkieGroundVelocity rate lim

Monitoring : intensity:0.08


[2024-11-30 17:29:12,990] [warning] UpkieGroundVelocity rate limiter is late by -0.600000 [ms] (rate_limiter.py:106)
[2024-11-30 17:29:15,600] [warning] UpkieGroundVelocity rate limiter is late by -3.500000 [ms] (rate_limiter.py:106)
[2024-11-30 17:29:15,774] [warning] UpkieGroundVelocity rate limiter is late by -0.800000 [ms] (rate_limiter.py:106)


Monitoring : intensity:0.09


IndexError: index 10 is out of bounds for axis 0 with size 10